<a href="https://colab.research.google.com/github/ayumu-1212/tabelog_analysis/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os
import csv
import uuid
from bs4 import BeautifulSoup
from datetime import datetime as dt
from google.colab import drive
drive.mount('/content/drive')

"""
【概要】
店舗ごとの画像フォルダ分け無し
ページ遷移あり
.
└── data
    ├── train_labels.csv
    └── train
        ├── abb65fac-2f52-433e-bcaa-0ef8e36dc0cf.jpg
        ├── c5aa2b2a-6f5c-45c7-bf42-8852e7654b55.jpg
        ├── (…)

【課題】
- scoreがなかった時の処理 -> 前処理で実装、pandas
- dev, testデータの作成 -> splitとかで実装可能
- データ数が多くなった時の処理 -> 実装済み（1000店舗分）
- 画像1枚1枚に対してスコアが現状 -> 将来的に画像3枚に対するスコアを回帰させることを検討
"""

HEADER = ['img_path','score']
MAX_PAGE = 100
MAX_SHOP = 1000
now_time_s = dt.now().strftime("%Y%m%d%H%M%S")
csv_path = "./drive/MyDrive/Colab Notebooks/CATechAccel/data/train_labels_" + now_time_s + ".csv"
new_imgdir_name = 'train_' + now_time_s
os.mkdir("./drive/MyDrive/Colab Notebooks/CATechAccel/data/" + new_imgdir_name)

with open(csv_path, "w", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(HEADER)
    shop_index = 0
    for n in range(1, MAX_PAGE+1):
        # ショップ数が十分か判定
        if shop_index >= MAX_SHOP:
            break
        print("{}ページ目開始".format(n))
        response = requests.get('https://tabelog.com/tokyo/rstLst/' + str(n) + "/")
        soup = BeautifulSoup(response.text, 'lxml')
        restaurant_boxes = soup.find_all("div", class_="list-rst__body")
        for box in restaurant_boxes:
            # ショップ数が十分か判定
            if shop_index >= MAX_SHOP:
                break

            # レストラン名取得
            name = box.find("div", class_="list-rst__contents")\
                .find("div", class_="list-rst__rst-data")\
                .find("div", class_="list-rst__rst-name-wrap").h3.a.text

            # レビュースコア取得
            try:
                score = box.find("div", class_="list-rst__contents")\
                    .find("div", class_="list-rst__rst-data")\
                    .find("div", class_="list-rst__rate").p.span.string
            except:
                print("スコアなし：{}".format(name))
                continue

            # サムネ画像3枚取得
            try:
                lis = box.find('div', class_='list-rst__rst-photo')\
                    .find('ul', class_="list-rst__thumb-list")\
                    .find_all('li', class_='list-rst__thumb-item')
            except:
                print("画像なし：{}".format(name))
                continue

            # サムネ画像保存 & csvデータ保存
            for li in lis:
                img = li.p.a.img
                r = requests.get(img['data-lazy'])
                file_name = str(uuid.uuid4())+str('.jpg')
                with open("./drive/MyDrive/Colab Notebooks/CATechAccel/data/" + new_imgdir_name + "/" + file_name, 'wb') as file:
                      file.write(r.content)
                writer.writerow([new_imgdir_name + "/" + file_name, score])

            shop_index += 1
    print("スクレイピング終了")

Mounted at /content/drive
1ページ目開始
2ページ目開始
3ページ目開始
画像なし：神楽坂 鮨 弥栄
4ページ目開始
5ページ目開始
6ページ目開始
スコアなし：全席個室 肉バル NIKUTOBA 新橋店
7ページ目開始
8ページ目開始
スコアなし：Beef street@Free Hill
9ページ目開始
10ページ目開始
11ページ目開始
12ページ目開始
13ページ目開始
14ページ目開始
15ページ目開始
16ページ目開始
17ページ目開始
スコアなし：無制限飲み放題 個室居酒屋 酔ってき屋 新宿店
18ページ目開始
19ページ目開始
スコアなし：250席全席扉付き完全個室居酒屋 わらやき道場 新橋駅前店
20ページ目開始
21ページ目開始
22ページ目開始
23ページ目開始
スコアなし：和牛ステーキ×食べ放題 個室完備のお店 MEAT TOKYO 新宿東口店
スコアなし：全席扉付き完全個室 鳥之助 新橋SL広場店
24ページ目開始
25ページ目開始
スコアなし：焼鳥 ワイン 星乃
26ページ目開始
27ページ目開始
28ページ目開始
29ページ目開始
30ページ目開始
31ページ目開始
スコアなし：九州蔵
32ページ目開始
33ページ目開始
34ページ目開始
35ページ目開始
36ページ目開始
37ページ目開始
スコアなし：焼肉少年團 渋谷店
38ページ目開始
39ページ目開始
40ページ目開始
スコアなし：食べ飲み放題全130種類 完全個室居酒屋 京園 新宿本店
41ページ目開始
スコアなし：個室居酒屋 九十九 新宿2号店
42ページ目開始
43ページ目開始
スコアなし：個室居酒屋 吟楽 有楽町駅前店
44ページ目開始
45ページ目開始
スコアなし：299円均一個室居酒屋 炭火焼き鳥 九十九 池袋西口店
46ページ目開始
47ページ目開始
48ページ目開始
49ページ目開始
50ページ目開始
51ページ目開始
スクレイピング終了
